In [ ]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

a = 3
s = 3
r = 3
rr = 20
ro = 70
ra = 300
noise = 0.01
prop = 0.4
weight = 3
bias = np.array([0.0,-1.0])
dev_bias = 3
dTheta = 120
sight_theta = 180


In [138]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    radius = height/2
    agents = [Point(width/2 + radius*random()*np.cos(2*np.pi*random()), #random in a circunference
                    height/2 + radius*random()*np.sin(2*np.pi*random())) for i in range(N)]
    #agents = [Point(width*random(), height*random()) for i in range(N)] #Random in whole window
    #agents = [Point(width*(0.5*random()), height*(0.25+0.5*random())) for i in range(N)]  #Random in a limited square 
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2)))
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        a, dists[i] = relative_pos(agent, agents[i])
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    #return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repultion radius.")
        

        
def virtualizer (current, agents, h, w, N):
    lower_limit = current.getY() - h / 2
    upper_limit = current.getY() + h / 2
    left_limit = current.getX() - w / 2
    right_limit = current.getX() + w / 2
    
    vagents=[np.array([agent.getX(),agent.getY()]) for agent in agents]
    virtuals = N * [0.0, 0.0]
    vvirtuals = N * [Point(0.0, 0.0)]

    for j in range(N):
        #make more compact!!!
        virtuals[j]=vagents[j]
        
        #newcopy = agents[i].clone()
        candidates = [vagents[j],
                      vagents[j]+[w,0],vagents[j]+[-w,0],vagents[j]+[w,h],vagents[j]+[w,-h],
                      vagents[j]+[-w,h],vagents[j]+[-w,-h],vagents[j]+[0,h],vagents[j]+[0,-h]]
        #print candidates
        
        #virtuals[j] = next((cand for cand in candidates if lower_limit < cand[1] < upper_limit and left_limit < cand[0] < right_limit),False
        for i in range(9):
            if lower_limit < candidates[i][1] < upper_limit and left_limit < candidates[i][0] < right_limit:
                virtuals[j]=candidates[i]
            
        ##ATTENTION HERE!!
#        if not virtuals[j]:
 #           virtuals[j] = agents[j]
        vvirtuals[j] = Point(virtuals[j][0],virtuals[j][1])
        
    return vvirtuals

        
#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return


In [139]:
#COUZIN MODEL IMPLEMENTED WITH REPULSE, ATRACT AND ORIENT ZONES SEPARATED (1ST PAPER)
def couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_range, model2, roa, atract, orient):
    
    if not model2:
        atract = orient = 1
    # watch only particles in repulsion zone
    for i in range(N):
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):     
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], virtuals[j])
            
            if in_sight_range(rel_pos, speeds[i], sight_range):
                if distance < rr:
                    rel_pos = normalized(rel_pos)
                    r_dir = r_dir + rel_pos
                    repulsion_flag = True

                elif not repulsion_flag:
                    
                    if model2: # Couzin 2
                        if distance < roa:
                            o_dir = o_dir + speeds[j]
                            rel_pos = normalized(rel_pos)
                            a_dir = a_dir + rel_pos

                    else: # Couzin 1
                        if distance < ro:
                            o_dir = o_dir + speeds[j]

                        elif distance < ra:
                            rel_pos = normalized(rel_pos)
                            a_dir = a_dir + rel_pos

        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(- r_dir)
        else:
            tot_dir = orient * normalized(o_dir) + atract * normalized(a_dir)
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noisy_vector(noise))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = softened_angle(speeds[i], tot_dir, s, dTheta)
    return

    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir
    return

##MILL MODEL
def mill (dt, agents, speeds, N, width, height, cr, ca, lr, la, alpha, beta):
    clr = cr / lr
    cla = ca / la
    for i in range(N):
        u_dir = np.array([0.0, 0.0])
        propulsion = np.array([0.0, 0.0])
        friction = np.array([0.0, 0.0])
        grad_U = np.array([0.0, 0.0])
        
        virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):   #Duality interactions, by the Morse potential  
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], virtuals[j])
            u_dir = normalized(rel_pos)
            grad_U = grad_U + u_dir * (clr*np.exp(-distance / lr) - cla *np.exp(- distance / la))
            
        propulsion = alpha * speeds[i] # self-propulsion propto alpha
        friction =  beta * ((normalized(speeds[i]))**2) * speeds[i] #friction force propto beta
        d_speed = propulsion - friction - grad_U
        speeds[i]= speeds[i]+dt*d_speed
        print u_dir
    
   # print speeds[N-1], propulsion, friction, grad_U
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    bias = np.array([0.0,-1.0])
    for i in range(N_steps):
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_theta, 1, 200, 1.5, 1) #(...)last 4 parameters: model2, roa, orient, atract
        #mill (dt, agents, speeds, N, width, height, cr=1.0, ca=0.5, lr=0.5, la=3.0, alpha=1.6, beta=0.5)
        #mill (dt, agents, speeds, N, width, height, cr=40, ca=5, lr=5 ,la=100.0, alpha=0.2, beta=0.5) #equilibrium for 5 particles
        mill (dt, agents, speeds, N, width, height, cr=40, ca=50, lr=5 ,la=100.0, alpha=0.3, beta=0.6) #almost equilibrium for 50 particles
        
        
        #interaction(agents, speeds)
        
        
        #biaser(speeds, N, s, i, prop, bias, dev_bias, weight)
        
        #INFORMATION TRANSFER: SHAPE & DIRECTION & ALIGNMENT QUALITY 
        #point_cm.undraw()
        #old_cm = quality(agents, speeds, N, bias, window, old_cm)
        
        ## BOUNDARY CONDITIONS
        rigid_boundary(width, height, agents, speeds, N)
        #periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return

In [ ]:
run(10000, 50, 5, 1, 900, 700)

[ 0.43265359  0.90156024]
[-0.84497229 -0.53481008]
[ 0.58314194 -0.81237029]
[ 0.95623223 -0.29260883]
[ 0.68061685 -0.73263955]
[ 0.42214041  0.90653046]
[-0.1545704   0.98798178]
[ 0.91697304  0.39894919]
[-0.93058776 -0.36606888]
[ 0.69881721 -0.71530029]
[ 0.87048995 -0.4921862 ]
[ 0.99985848 -0.016823  ]
[ 0.94007212  0.34097567]
[ 0.29372424 -0.9558902 ]
[ 0.95391266 -0.30008437]
[ 0.84888531  0.52857708]
[-0.56759797 -0.82330586]
[ 0.98650021  0.16376001]
[ 0.99685663 -0.0792266 ]
[ 0.95419733  0.29917795]
[ 0.7996989  -0.60040125]
[ 0.423422   -0.90593256]
[ 0.99991239 -0.01323652]
[ 0.98759079  0.15704915]
[-0.7893906   0.61389126]
[ 0.92623061 -0.37695737]
[ 0.46565196 -0.88496794]
[ 0.70277318 -0.71141399]
[ 0.30456737 -0.9524908 ]
[ 0.86973694 -0.49351561]
[ 0.19339604 -0.98112077]
[ 0.51369436 -0.85797325]
[ 0.9786795   0.20539336]
[ 0.5968017   0.80238877]
[ 0.49041112 -0.87149121]
[ 0.92907001  0.36990393]
[-0.0562928 -0.9984143]
[ 0.5131002  -0.85832872]
[ 0.99653465 -

[ 0.91915157  0.39390403]
[-0.83313092  0.55307583]
[ 0.94638648 -0.32303658]
[ 0.43931583 -0.89833268]
[ 0.71330398 -0.70085479]
[ 0.11817831 -0.99299239]
[ 0.9170325  -0.39881248]
[ 0.03963043 -0.99921441]
[ 0.43240167 -0.90168109]
[ 0.95614     0.29291005]
[ 0.43088051  0.90240899]
[ 0.34617376 -0.93817042]
[-0.241632    0.97036796]
[-0.34102853 -0.94005295]
[ 0.35947559 -0.93315449]
[ 0.99855356  0.05376612]
[ 0.91750582 -0.39772236]
[ 0.98951982  0.14439711]
[ 0.98877553 -0.14940867]
[ 0.51554761  0.85686093]
[ 0.84051004 -0.54179597]
[ 0.96297386 -0.26959476]
[ 0.9755115  -0.21994842]
[-0.23054846  0.97306085]
[ 0.98710092  0.16009929]
[ 0.68556652  0.72800999]
[-0.68556652 -0.72800999]
[ 0.25743937  0.96629445]
[-0.98098452 -0.19408599]
[ 0.46032783 -0.887749  ]
[ 0.99995381  0.00961154]
[ 0.59537002 -0.80345164]
[-0.01801193  0.99983777]
[-0.35374619 -0.93534145]
[ 0.56024816  0.82832482]
[ 0.96057462 -0.27802231]
[ 0.60100932 -0.79924201]
[ 0.91912887 -0.393957  ]
[ 0.99244659

[ 0.7846759   0.61990622]
[ 0.99806085  0.06224587]
[ 0.13691148  0.99058329]
[-0.60492927  0.79627921]
[-0.48670481  0.8735665 ]
[-0.34319686  0.9392635 ]
[-0.51340728 -0.85814507]
[-0.78845942  0.61508678]
[ 0.19591309  0.98062126]
[-0.19591309 -0.98062126]
[ 0.021821    0.99976189]
[-0.99152945  0.12988206]
[-0.91576316 -0.4017186 ]
[-0.69112718  0.72273316]
[ 0.24536193 -0.96943155]
[-0.28664601  0.95803657]
[-0.68615854 -0.72745204]
[-0.57629779  0.81723978]
[ 0.99555239  0.0942095 ]
[-0.89266535 -0.45072006]
[-0.58538083  0.81075846]
[-0.91863393  0.39510973]
[-0.86291308  0.50535237]
[-0.25791683  0.96616712]
[-0.58328638  0.81226658]
[-0.45079863  0.89262567]
[-0.70237252  0.71180956]
[ 0.705453   0.7087567]
[ 0.77875941  0.62732271]
[ 0.44499705  0.89553203]
[-0.6130738   0.79002564]
[-0.61899285 -0.78539662]
[ 0.74654077  0.66533968]
[-0.70909452  0.70511344]
[-0.8226465  0.5685532]
[-0.48022218  0.87714688]
[-0.22855481  0.97353105]
[-0.67668285  0.73627462]
[-0.93772245 -0.

[ 0.29412688  0.95576638]
[-0.59798068 -0.80151051]
[ 0.63043076 -0.77624549]
[ 0.90887095  0.41707745]
[-0.79613339  0.60512117]
[-0.46121105  0.88729047]
[ 0.03565245  0.99936425]
[-0.99961381 -0.02778909]
[-0.79264993  0.60967704]
[-0.63329526  0.77391027]
[-0.99845606 -0.05554728]
[ 0.05734008 -0.9983547 ]
[-0.45348631 -0.89126324]
[ 0.87294235  0.48782339]
[-0.87394902 -0.4860176 ]
[ 0.74684717 -0.66499572]
[-0.19131938  0.98152784]
[ 0.87929893  0.4762703 ]
[-0.91616898 -0.40079222]
[-0.84323589  0.53754371]
[-0.48714352  0.87332193]
[-0.4333252   0.90123763]
[ 0.87749006  0.47959482]
[ 0.60189825  0.79857279]
[ 0.99136059  0.13116468]
[ 0.71892912  0.6950834 ]
[-0.60484431  0.79634374]
[ 0.5205151   0.85385246]
[ 0.20118035  0.97955422]
[ 0.72321622  0.69062168]
[-0.39125074  0.92028412]
[ 0.31071001  0.95050475]
[ 0.20933702  0.97784355]
[-0.88466215 -0.46623265]
[-0.94464468  0.32809514]
[-0.71235114 -0.70182323]
[ 0.97064142 -0.24053113]
[ 0.8849197   0.46574362]
[ 0.93131909

[ 0.71259726 -0.70157334]
[ 0.55874637 -0.82933859]
[ 0.79471879 -0.60697779]
[ 0.24686439 -0.96905004]
[ 0.81080208 -0.58532041]
[-0.025825   -0.99966648]
[ 0.025825    0.99966648]
[ 0.9997763  -0.02115053]
[-0.91943409 -0.39324415]
[ 0.4342907  -0.90077277]
[ 0.69966533 -0.71447073]
[-0.38358501 -0.92350557]
[ 0.82103465 -0.57087837]
[-0.33122942 -0.94355025]
[ 0.29614553 -0.95514283]
[ 0.54238958  0.8401271 ]
[ 0.69330799 -0.7206414 ]
[ 0.71601907 -0.69808072]
[ 0.31971525 -0.94751367]
[ 0.98644817  0.1640732 ]
[ 0.69635827 -0.71769434]
[ 0.86252991 -0.50600608]
[ 0.84353388 -0.53707596]
[ 0.69099007 -0.72286425]
[ 0.13597245 -0.99071262]
[-0.25160628 -0.96782967]
[-0.22635841 -0.97404408]
[ 0.83311329 -0.55310238]
[ 0.90764767 -0.4197329 ]
[-0.25838753 -0.96604135]
[ 0.997963    0.06379539]
[-0.88767893  0.46046295]
[ 0.72095821 -0.69297854]
[ 0.71218927 -0.70198749]
[ 0.5574694  -0.83019749]
[ 0.02075144 -0.99978467]
[ 0.71128266 -0.7029061 ]
[ 0.12301937 -0.99240427]
[ 0.7796277 

...

[ 0.93198287 -0.36250233]
[ 0.67680956  0.73615815]
[ 0.63030932 -0.7763441 ]
[ 0.72644804  0.68722139]
[ 0.81851838  0.57448034]
[ 0.63732688  0.77059357]
[ 0.61112652  0.79153293]
[-0.58287592  0.81256117]
[ 0.99888541 -0.04720099]
[-0.73387016 -0.67928977]
[ 0.84884576 -0.52864059]
[-0.94042992  0.33998759]
[ 0.68134875  0.73195893]
[ 0.99779236  0.06641086]
[-0.96188416 -0.27345725]
[-0.65183593  0.75836002]
[ 0.60783771 -0.79406128]
[ 0.98884698  0.14893507]
[ 0.68283262  0.73057485]
[ 0.82761027  0.56130317]
[ 0.40151054  0.9158544 ]
[-0.97217538 -0.23425422]
[ 0.76564956  0.64325792]
[ 0.73713958  0.67574051]
[ 0.98182458  0.18979065]
[ 0.97448912 -0.22443476]
[ 0.40716465  0.91335478]
[ 0.69229532  0.72161429]
[ 0.28379125 -0.95888609]
[ 0.99991394  0.01311921]
[ 0.32708875  0.94499362]
[ 0.94386331 -0.33033627]
[ 0.86893694  0.49492281]
[ 0.75732219  0.65304142]
[ 0.75251417  0.65857606]
[-0.50720592 -0.8618249 ]
[ 0.72267951  0.69118328]
[ 0.72841849  0.68513248]
[ 0.77578427

[-0.99996449  0.00842668]
[-0.6395891  -0.76871697]
[ 0.08836379 -0.99608827]
[-0.90751901  0.420011  ]
[ 0.69806586  0.71603356]
[-0.81396192 -0.58091823]
[-0.43713445 -0.89939617]
[-0.53388738 -0.84555559]
[-0.62335464 -0.78193925]
[-0.99995333 -0.0096607 ]
[ 0.33931872 -0.94067146]
[-0.2350983 -0.9719716]
[-0.71810605 -0.69593369]
[-0.92040923 -0.39095632]
[ 0.28615338 -0.95818383]
[ 0.68955602  0.72423235]
[-0.95593424 -0.29358086]
[ 0.93546176 -0.35342793]
[ 0.79724713  0.60365306]
[-0.15659846 -0.98766235]
[-0.65161966 -0.75854586]
[-0.6583392 -0.7527214]
[-0.53046601 -0.84770621]
[ 0.26342316  0.96468038]
[ -3.24954018e-05  -9.99999999e-01]
[-0.1399673  -0.99015613]
[-0.2233026  -0.97474917]
[-0.51482477 -0.85729543]
[-0.96778725 -0.2517694 ]
[-0.97663253 -0.21491605]
[-0.65578546 -0.7549473 ]
[ 0.63093561 -0.7758352 ]
[-0.40213612 -0.9155799 ]
[-0.16540196 -0.98622624]
[-0.36213896 -0.93212412]
[-0.65318719 -0.75719647]
[-0.57121133 -0.82080303]
[-0.42915068 -0.90323291]
[-0.61

[ 0.78917922  0.61416297]
[ 0.85527147  0.5181802 ]
[ 0.90614644  0.4229641 ]
[ 0.8119473   0.58373074]
[-0.65387278 -0.75660451]
[-0.64144135 -0.76717208]
[ 0.92047112  0.3908106 ]
[-0.92047112 -0.3908106 ]
[ 0.96568404  0.25971974]
[ 0.84373504  0.53675989]
[ 0.7884398   0.61511193]
[ 0.78865501  0.61483598]
[ 0.72153877  0.69237403]
[ 0.98586104  0.16756496]
[-0.55484347 -0.83195476]
[ 0.99525459 -0.09730516]
[ 0.76772347  0.6407813 ]
[ 0.20767228  0.97819846]
[-0.15791687  0.98745241]
[-0.80343221 -0.59539624]
[ 0.89034954  0.4552776 ]
[ 0.87118773  0.49095003]
[ 0.71714068  0.69692844]
[ 0.61503781  0.78849762]
[ 0.99941127 -0.03430905]
[ 0.96623032  0.25767996]
[ 0.73073025  0.68266632]
[ 0.81328831  0.58186091]
[ 0.70905078  0.70515742]
[ 0.8539837   0.52029977]
[-0.59945629 -0.8004075 ]
[ 0.96677283  0.25563706]
[ 0.99504165 -0.09945911]
[-0.74181654  0.67060288]
[ 0.72744823  0.68616257]
[ 0.87293926  0.4878289 ]
[ 0.75526576  0.65541867]
[ 0.74669388  0.66516783]
[ 0.31231365

[-0.66700751 -0.74505099]
[-0.47072719 -0.88227882]
[-0.64364264 -0.76532617]
[-0.29039787 -0.95690599]
[-0.98475756  0.17393258]
[-0.67235366 -0.74023007]
[-0.97212353 -0.23446927]
[-0.90827624 -0.41837098]
[-0.00515428 -0.99998672]
[ 0.34245465 -0.93953436]
[ 0.85883235  0.51225677]
[-0.64054739 -0.76791864]
[-0.93593173 -0.35218147]
[-0.71033422 -0.70386455]
[-0.63287773 -0.77425175]
[-0.97989099  0.19953356]
[-0.96371068  0.26694892]
[-0.74792912 -0.66377861]
[ 0.68373111  0.72973404]
[-0.5742259 -0.8186969]
[-0.51997135 -0.8541837 ]
[-0.70174205 -0.71243112]
[-0.63496634 -0.77253981]
[ 0.96167357 -0.2741969 ]
[-0.95406454  0.29960116]
[-0.61808394 -0.78611211]
[-0.62740807 -0.77869064]
[-0.58489369 -0.81110996]
[-0.66065056 -0.75069357]
[-0.97560254 -0.21954425]
[-0.61684276 -0.78708641]
[-0.51417843 -0.85768324]
[-0.51716816 -0.85588381]
[-0.69612919 -0.71791654]
[-0.71420316 -0.69993846]
[-0.82079263 -0.57122627]
[-0.65289014 -0.75745262]
[-0.58646271 -0.80997622]
[-0.70578812 -

[-0.7797741  -0.62606098]
[-0.80410768 -0.59448368]
[-0.67745642 -0.73556291]
[-0.65490562 -0.75571068]
[-0.97923677 -0.20271987]
[-0.8210014  -0.57092618]
[-0.85857418 -0.51268935]
[-0.33325319 -0.94283737]
[-0.69576753 -0.71826704]
[-0.82428888 -0.56616945]
[-0.48197896 -0.87618279]
[-0.7210588  -0.69287388]
[ 0.7207547  0.6931902]
[-0.99601082 -0.08923257]
[ 0.99601082  0.08923257]
[-0.99792984 -0.06431196]
[-0.82084931  0.57114483]
[-0.70332625 -0.71086721]
[ 0.73532536  0.67771426]
[-0.89763005  0.44074969]
[ 0.71062946 -0.70356646]
[ 0.71363297  0.7005198 ]
[-0.75504145  0.65567707]
[ 0.7652846  -0.64369207]
[-0.68960156  0.72418898]
[ 0.7534616  -0.65749192]
[ 0.95310086  0.30265287]
[ 0.42730114  0.90410936]
[ 0.35418474 -0.93517548]
[ 0.0948258   0.99549388]
[ 0.28051414  0.95984989]
[ 0.62090722 -0.78388406]
[ 0.80303709 -0.59592905]
[-0.10525468  0.9944453 ]
[ 0.69876043  0.71535576]
[ 0.80772512  0.58955927]
[-0.9038425   0.42786532]
[-0.70941687 -0.70478912]
[-0.89101676 -

KeyboardInterrupt: 